![図1.2 Sparkの紹介とインストール](images/sparkBasics.png)

# 本セクションの目次
1. Spark Sessionとは？
2. Sparkを用いたバッチにおけるエンジニアリングの流れ(データソース、変換、カラムナーフォーマット、テーブル作成)？
3. データソースの読み込み
4. データ変換
5. DIKWモデル
6. カラムナーフォーマットへの変換
7. スモールファイル問題
8. 参照用のテーブル作成

# Spark Sessionとは？

Javaで言うところのインスタンスを作る作業のことです(new Class())。  
今回の場合は、アプリケーション名が「chapter2」で作成を行っています。  

```
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は次のチャプターで説明を行います。
spark = SparkSession.builder \
    .appName("chapter2") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()
```

configの部分で  
非常に細かい設定ができるので、詳しくは公式のドキュメントを参考にしてください https://spark.apache.org/docs/3.1.1/  
一部メモリの設定いついては「Sparkを本番環境で動かす」チャプターにて紹介します

# Sparkを用いたバッチにおけるデータエンジニアリング一連の流れ

1. データソースの読み込み(今回は、人口統計データ(/dataset/jinko.csv))　ETL(Extract Transform Load)で言うEの部分
2. 変換を行う(集計等を行う)　DataFrame処理 or SQL処理の２パターンで実行可能 ETL(Extract Transform Load)で言うTの部分
3. カラムナーフォーマットへ変換する ETL(Extract Transform Load)で言うTの部分
4. 出力したデータをみんなに見やすくするため(BIツールから参照できるように)テーブルを作成する ETL(Extract Transform Load)で言うLの部分

よくある、関数の羅列をするのではなく、実業務に沿った形で流れを紹介していきます。

## データソース
データの源。リレーショナルデータベースのときもあれば、今回のようにファイルの形式のときもある。  
更に進むと、PDFやEXCELなんて事もあります。  
ストリーミングだとIoTであったり、Webブラウザのアクセスログだったりとデータになりうるものは無限に存在しています。  

## 変換処理
ETL（Extract Transform Load）というと少し定義として広いのかもしれないのですが、  
データを整形してより分析向けの形(フォーマット変換や圧縮含む)にしたり、精度の高いデータを作成する行為のことです。  
そのため、ETLというとバッチ処理のイメージを持つ人も多いかもしれませんが、ストリーミングデータにも適用される言葉です。   
Tの処理はSparkにおいてDataFrameもしくはSQLで処理することができる(RDDと呼ばれるものもあるが、労力の割に実際は出番はあまりなく今回は取り扱わない)

## カラムナーフォーマットへ変換を行う
ビッグデータの世界では、Apache Parquet と呼ばれるフォーマットが広く使われています。  
CSV形式のようなローフォーマットはビックデータ処理において処理効率が悪いため、早い段階でParquetに変換を行います。  
分析用のSQLの実行であったり、複数台で処理することに向いているフォーマットです。  

Parquetの特徴としては以下になります。

- カラムナー（ストレージ）フォーマット
- カラムごとに圧縮が効くため、効率よくデータをストアできる
- 多くのプロダクトがサポートしている

多くのプロダクトはParquetを取り込んだり処理したりする機能を提供してくれており単体ではなく総合で使えるフォーマットです。

## テーブル形式での保存

多くは、実データとテーブル定義が分離された`ロケーション方式`をとっている。  
後ほど実際に作成してみますが、イメージは以下のような感じです。

```
CREATE EXTERNAL TABLE IF NOT EXISTS sample.sampletable ( id INT, date STRING)
PARTITIONED BY (dt INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '/Users/yuki/pyspark_batch/dataset/parquet/';

#S3などであれば、以下のように設定を変えることも可能です。
LOCATION 's3://data.platform/sample.db/raw_zone/sampletable/';


```

テーブルとして保存することによって、非エンジニアにも扱いやすくしてデータを提供することが可能です。

In [1]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
spark = SparkSession.builder \
    .appName("chapter2") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

21/10/11 13:17:04 WARN Utils: Your hostname, saitouyuuki-no-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.10 instead (on interface en0)
21/10/11 13:17:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/11 13:17:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# データソースの読み込み

In [2]:
#データソースの読み込み
#sep='\t'とすればtsvでも読み込みが可能です
#multiLineは、CSVやTSVの各カラムに改行が含まれていた時の対策です。
df=spark.read.option("multiLine", "true").option("encoding", "SJIS").csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=True, sep=',', inferSchema=False)
df.count()

982

In [3]:
df.show(truncate=False)
#truncate Falseで表示結果を省略せず表示します。

+--------------+----------+----+----------+----------+----+------------+----------+----------+
|都道府県コード|都道府県名|元号|和暦（年）|西暦（年）|注  |人口（総数）|人口（男）|人口（女）|
+--------------+----------+----+----------+----------+----+------------+----------+----------+
|00            |全国      |大正|9         |1920      |null|55963053    |28044185  |27918868  |
|01            |北海道    |大正|9         |1920      |null|2359183     |1244322   |1114861   |
|02            |青森県    |大正|9         |1920      |null|756454      |381293    |375161    |
|03            |岩手県    |大正|9         |1920      |null|845540      |421069    |424471    |
|04            |宮城県    |大正|9         |1920      |null|961768      |485309    |476459    |
|05            |秋田県    |大正|9         |1920      |null|898537      |453682    |444855    |
|06            |山形県    |大正|9         |1920      |null|968925      |478328    |490597    |
|07            |福島県    |大正|9         |1920      |null|1362750     |673525    |689225    |
|08            |茨城県    |大正|9       

# 変換を行う(集計等を行う)

In [6]:
# ここからはETLにおけるTを行っていきます

#大正や昭和はもう不要かなと感じたら変換処理にて
df_after_t=df.where(df."和暦（年）"== "平成")

SyntaxError: invalid syntax (1624560064.py, line 4)

In [9]:
#うーん使いづらい。。(日本語))
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

#スキーマ設定をしていきましょう
# カラム名、型、NullOKか？で設定していきます
struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "SJIS") \
    .csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)
df.show()

+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|          code|    kenmei|gengo|    wareki|   seireki| chu|       sokei|jinko_male|jinko_female|
+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|都道府県コード|都道府県名| 元号|和暦（年）|西暦（年）|  注|人口（総数）|人口（男）|  人口（女）|
|            00|      全国| 大正|         9|      1920|null|    55963053|  28044185|    27918868|
|            01|    北海道| 大正|         9|      1920|null|     2359183|   1244322|     1114861|
|            02|    青森県| 大正|         9|      1920|null|      756454|    381293|      375161|
|            03|    岩手県| 大正|         9|      1920|null|      845540|    421069|      424471|
|            04|    宮城県| 大正|         9|      1920|null|      961768|    485309|      476459|
|            05|    秋田県| 大正|         9|      1920|null|      898537|    453682|      444855|
|            06|    山形県| 大正|         9|      1920|null|      968925|    478328|      49059

In [10]:
# 大正や昭はもう不要かなと感じたら変換処理にて
df.where(df.gengo == "平成").show()
# where 以外にもfilterと呼ばれるものがあります。機能は同じなので好きな方を選んで大丈夫です
df.filter(df.gengo == "平成").show()

+----+----------------------+-----+------+-------+----+---------+----------+------------+
|code|                kenmei|gengo|wareki|seireki| chu|    sokei|jinko_male|jinko_female|
+----+----------------------+-----+------+-------+----+---------+----------+------------+
|  00|                  全国| 平成|     2|   1990|null|123611167|  60696724|    62914443|
|  0A|          人口集中地区| 平成|     2|   1990|null| 78152452|  38564229|    39588223|
|  0B|人口集中地区以外の地区| 平成|     2|   1990|null| 45458715|  22132495|    23326220|
|  01|                北海道| 平成|     2|   1990|null|  5643647|   2722988|     2920659|
|  02|                青森県| 平成|     2|   1990|null|  1482873|    704758|      778115|
|  03|                岩手県| 平成|     2|   1990|null|  1416928|    680197|      736731|
|  04|                宮城県| 平成|     2|   1990|null|  2248558|   1105103|     1143455|
|  05|                秋田県| 平成|     2|   1990|null|  1227478|    584678|      642800|
|  06|                山形県| 平成|     2|   1990|null|  1258390|

In [11]:
#集計をしてみます
#平成の県ごとの男女の数の平均
#groupByは県名ごとにグルーピングする記述です。
#aggはカラムごとに集計する関数で今回は男性の人口と、女性の人口毎の平均を集計しています。
#aliasは別名をつける関数です。例えば、女性の人口毎の平均の結果にはfemale_avgという別名を付けています。
import pyspark.sql.functions as sf
df.where(df.gengo == "平成").groupBy("kenmei") \
  .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")).show()

+----------------------+--------------------+--------------------+
|                kenmei|            male_avg|          female_avg|
+----------------------+--------------------+--------------------+
|人口集中地区以外の地区|2.0976203166666668E7|2.2272045166666668E7|
|                佐賀県|            408192.5|            456442.5|
|                栃木県|   987741.8333333334|            999415.5|
|                京都府|  1268325.3333333333|  1360099.3333333333|
|                香川県|   485871.8333333333|   523763.6666666667|
|                愛媛県|   692188.3333333334|   774376.1666666666|
|                秋田県|   542928.3333333334|            604578.5|
|                広島県|           1387308.5|  1478006.8333333333|
|                宮崎県|            542386.5|            608793.0|
|              鹿児島県|            818506.0|            929134.0|
|                埼玉県|  3492880.3333333335|  3443447.8333333335|
|                三重県|            893167.5|   944959.6666666666|
|                島根県|            356034.

# このような集計をクロス集計と呼びます
クロス集計とは複数の項目をかけ合わせて集計を行うことです。
今回の場合は、県名と男女をかけ合わせてそれぞれの県別、男女別のクロス集計を行いました。

逆に、県名だけの集計などは単純集計と呼ばれています。

# データクレンジング
データを操作していると、時には不要と感じる事もあります。 
不要でデータを除外したり、並び替えたりより精度の高いデータにすることをデータクレンジングと呼びます。

In [12]:
#「人口集中地区以外の地区」がいらなそうですね。
# データをクレンジングして不要なデータを除きましょう
# sortは並び替えです（デフォルトでは昇順になります）

df.where(df.gengo == "平成").groupBy("kenmei") \
  .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")) \
    .filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg").show()

#良さそうです！

+--------+-----------------+-----------------+
|  kenmei|         male_avg|       female_avg|
+--------+-----------------+-----------------+
|  鳥取県|287885.3333333333|314291.3333333333|
|  島根県|         356034.5|388621.6666666667|
|  高知県|372268.1666666667|         418517.0|
|  徳島県|383399.1666666667|         423152.0|
|  福井県|         395512.5|420182.6666666667|
|  佐賀県|         408192.5|         456442.5|
|  山梨県|425777.8333333333|441831.1666666667|
|  香川県|485871.8333333333|523763.6666666667|
|和歌山県|         490624.0|547112.3333333334|
|  富山県|         532857.0|573049.8333333334|
|  宮崎県|         542386.5|         608793.0|
|  秋田県|542928.3333333334|         604578.5|
|  石川県|         566064.0|         604518.5|
|  大分県|571530.6666666666|638773.6666666666|
|  山形県|         583603.5|627811.6666666666|
|  沖縄県|         654622.0|679050.6666666666|
|  岩手県|659592.6666666666|714973.1666666666|
|  滋賀県|         662391.0|         680326.0|
|  奈良県|671178.6666666666|734736.6666666666|
|  青森県|675238.6666666666

In [13]:
#結果を一度保存しておきます
df_after_t=df.where(df.gengo == "平成").groupBy("kenmei") \
  .agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")) \
  .filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg")

# DIKWモデル
![図1.2 DIKW](images/DIKW.png)
少し脇道にそれるのですが、上記の作業はDIKWモデルというものに沿った動きでです。

DIKWモデルでは、データのステージを「Data」「Infromation」「Knowledge」「Wisdom」として定義しています。

- Data(データ)
- Information（情報）
- Knowledge（知識）
- Wisdom（知恵）

これらの頭文字をとってDIKWモデルと呼ばれています。

ETLをすることはDataを情報や知識に変換することを指します。
情報や知識は、データから見つかるルールや関係性のことです。
今回の場合だと、鳥取県が人口少ないです　という事実がわかったという形になります。

知恵は、この知識から生み出すもので、例えば鳥取県の人口が少なく、それが問題なのであれば
その問題を解決するための施策が知恵になります。



# カラムナーフォーマットへ変換する
データの変換が終わったので次は、そのデータをビッグデータ向けのフォーマットで保存することを考えます。  
今回はParquet形式へデータを変換します。


In [14]:
#単純に吐き出す方法
df_after_t.write.mode("overwrite").parquet("/Users/yuki/pyspark_batch/dataset/parquet")

21/10/11 17:06:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [15]:
#ファイルを見てみます
!ls -al /Users/yuki/pyspark_batch/dataset/parquet

#　確かにParquetnoファイルが出力されていますが、ファイルが多いですね。。 一つのファイルサイズが小さいのも気になります

total 792
drwxr-xr-x  102 yuki  staff  3264 Oct 11 17:07 .
drwxr-xr-x    4 yuki  staff   128 Oct 11 17:06 ..
-rw-r--r--    1 yuki  staff     8 Oct 11 17:07 ._SUCCESS.crc
-rw-r--r--    1 yuki  staff    16 Oct 11 17:06 .part-00000-2ec6caaf-10ee-4688-a37f-71779e20c13a-c000.snappy.parquet.crc
-rw-r--r--    1 yuki  staff    16 Oct 11 17:06 .part-00001-2ec6caaf-10ee-4688-a37f-71779e20c13a-c000.snappy.parquet.crc
-rw-r--r--    1 yuki  staff    16 Oct 11 17:06 .part-00002-2ec6caaf-10ee-4688-a37f-71779e20c13a-c000.snappy.parquet.crc
-rw-r--r--    1 yuki  staff    16 Oct 11 17:06 .part-00003-2ec6caaf-10ee-4688-a37f-71779e20c13a-c000.snappy.parquet.crc
-rw-r--r--    1 yuki  staff    16 Oct 11 17:06 .part-00004-2ec6caaf-10ee-4688-a37f-71779e20c13a-c000.snappy.parquet.crc
-rw-r--r--    1 yuki  staff    16 Oct 11 17:06 .part-00005-2ec6caaf-10ee-4688-a37f-71779e20c13a-c000.snappy.parquet.crc
-rw-r--r--    1 yuki  staff    16 Oct 11 17:06 .part-00006-2ec6caaf-10ee-4688-a37f-71779e20c13a-c000.snappy.pa

# スモールファイル問題
オンプレでもクラウドでもそうなのですが、ビッグデータの世界では一つのファイルが小さすぎると途端に処理が遅くなります。  
この問題をスモールファイル問題と読んでいます。  
一般に、１GBくらいずつまとめることが推奨されています。

この問題を解決するためには、repartition(もしくはcolaese)を使ってファイルをマージする必要があります。

In [16]:
# 今回はファイルを一個に纏めてみようと思います。
df_after_t.repartition(1).write.mode("overwrite").parquet("/Users/yuki/pyspark_batch/dataset/parquet")


In [17]:
# もう一度ファイルを見てみます
!ls -l /Users/yuki/pyspark_batch/dataset/parquet
# 一つになりました！ スモールファイル問題も解決です。
#　今回は一個にまとめましたが、一個にまとめるのが常に最適とは限りません。一つのサイズは１G~５GBくらいにするといいと言われています。

total 8
-rw-r--r--  1 yuki  staff     0 Oct 11 17:12 _SUCCESS
-rw-r--r--  1 yuki  staff  1990 Oct 11 17:12 part-00000-5a6c078e-ff20-44c8-96a5-276c2ac5eee0-c000.snappy.parquet


In [18]:
#もう少し書き込みのオプションを見ていきます
# partitionByを使うことで、データをパーティションごと(次に説明します)に分けて配置することができます。
# 今回はkenmei(県名)ごとにデータを保存してみようと思います。
df_after_t.repartition(1).write.partitionBy("kenmei").mode("overwrite").parquet("/Users/yuki/pyspark_batch/dataset/parquet")

In [20]:
# もう一度ファイルを見てみます
!ls -l /Users/yuki/pyspark_batch/dataset/parquet

#県名が出てきました
#三重県のデータはkenmei=三重県の下に格納されています。

total 0
-rw-r--r--  1 yuki  staff    0 Oct 11 17:14 _SUCCESS
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=三重県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=京都府
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=人口集中地区
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=佐賀県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=全国
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=兵庫県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=北海道
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=千葉県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=和歌山県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=埼玉県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=大分県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=大阪府
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=奈良県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=宮城県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=宮崎県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=富山県
drwxr-xr-x  4 yuki  staff  128 Oct 11 17:14 kenmei=山口県
d

In [21]:
#　三重県のデータを見てみます
!ls -l /Users/yuki/pyspark_batch/dataset/parquet/kenmei=三重県

total 8
-rw-r--r--  1 yuki  staff  741 Oct 11 17:14 part-00000-4be275c5-908f-4d9b-9ea7-d337a576284b.c000.snappy.parquet


In [22]:
# ここでSparkでParquetのデータを読み込んでみます

parquet_df=spark.read.parquet("/Users/yuki/pyspark_batch/dataset/parquet/kenmei=三重県")
parquet_df.show()

#コレが三重県の男性と女性の人口の平均値です。

+--------+-----------------+
|male_avg|       female_avg|
+--------+-----------------+
|893167.5|944959.6666666666|
+--------+-----------------+



# 出力したデータをみんなに見やすくするため(BIツールから参照できるように)テーブルを作成する
次はテーブルの作成を行ってみようと思います。  
今のままだとエンジニア向けでちょっと使い勝手が悪いのとBIツールといった他のツールから参照することができません

In [29]:
# Createテーブルを発行します
# 次のチャプターでも紹介しますが、spark.sqlという関数を使います

#jinko_avgテーブルを作成します。
#パーティションとはデータを分けるフォルダみたいなもの、パーティションを分けることで読み込むデータ量を少なくしたりできるので最適化できる
#先程確認したkenmei=の部分がパーティションになっているのでその出力結果に合わせてテーブルを作成してみます。

#ロケーションは、kenmeiを含まずに指定します。

spark.sql(""" 
CREATE EXTERNAL TABLE IF NOT EXISTS default.jinko_avg ( male_avg double, female_avg double)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/yuki/pyspark_batch/dataset/parquet/';
""")


DataFrame[]

In [25]:
# テーブルを見てみます。
spark.sql("show tables").show()

# ちゃんとできているようですね。
# ちなみに実行しているSQLは実はSQLみたいなものでHiveSQLと呼ばれるものです。
# Mysqlの扱いとほとんど同じなので、Mysqlみたいに使って動かなかったところだけ検索すると効率が良いと思います。

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
| default| jinko_avg|      false|
| default|jinko_avg2|      false|
+--------+----------+-----------+



In [30]:
#spark.sql()はdataframeを戻り値として返してくれます

df_result=spark.sql("select * from default.jinko_avg")
df_result.show()

#おや。データを見ることができません。。

+--------+----------+------+
|male_avg|female_avg|kenmei|
+--------+----------+------+
+--------+----------+------+



In [31]:
# テーブルだけでなく、Partitionを認識させてあげないといけません
# msck repair table　テーブル名と実行するとパーティションが認識されます(ちなみにAdd partitionというコマンドもあります)。
spark.sql("msck repair table jinko_avg")

DataFrame[]

In [32]:
# 今一度検索をしてみます。
spark.sql("select * from default.jinko_avg").show()
#　今度は出ましたね！

+--------------------+--------------------+------------+
|            male_avg|          female_avg|      kenmei|
+--------------------+--------------------+------------+
|            893167.5|   944959.6666666666|      三重県|
|  1268325.3333333333|  1360099.3333333333|      京都府|
|4.0840519833333336E7|4.2415789666666664E7|人口集中地区|
|            408192.5|            456442.5|      佐賀県|
|         6.1816723E7|6.4687834833333336E7|        全国|
|           2650310.5|           2861527.0|      兵庫県|
|  2665781.3333333335|  2923371.8333333335|      北海道|
|           2987847.0|           2974638.5|      千葉県|
|            490624.0|   547112.3333333334|    和歌山県|
|  3492880.3333333335|  3443447.8333333335|      埼玉県|
|   571530.6666666666|   638773.6666666666|      大分県|
|   4292675.833333333|           4517115.0|      大阪府|
|   671178.6666666666|   734736.6666666666|      奈良県|
|           1139561.5|           1191255.0|      宮城県|
|            542386.5|            608793.0|      宮崎県|
|            532857.0|

In [33]:
# もちろんSQLなので whereも可能です
spark.sql("select * from default.jinko_avg where kenmei='東京都'").show()

+-----------------+-----------------+------+
|         male_avg|       female_avg|kenmei|
+-----------------+-----------------+------+
|6222455.666666667|6268299.166666667|東京都|
+-----------------+-----------------+------+



# 次のチャプターはSparkSQLについて紹介していこうと思います

In [34]:
# Spark利用の停止
spark.stop()
spark.sparkContext.stop()